In [42]:
import os
import img2pdf
from unidecode import unidecode
import pandas as pd
from pdfconverter import convertword
import pdfconverter
from PIL import Image
import subprocess

In [43]:
def loop_miterd_criteria(main_folder, depth=0, df_logs=None, max_length=50, max_size_Mb=6):
    
    if df_logs is None:
        df_logs = {
            'errors': pd.DataFrame(columns=['path', 'message']),
            'Warning': pd.DataFrame(columns=['path', 'message'])
        }

    folders = [os.path.join(main_folder, c) for c in os.listdir(main_folder)]

    for path in folders:
        old_path = path
        path = unidecode(old_path)
        path_ = os.path.dirname(path)
        name, ext = os.path.splitext(os.path.basename(path))
        name = name.replace(" ", "_")
        if "Propuesta_" in name and os.path.basename(path_) in name:
            name_parts = name.split('_')
            if len(name_parts) > 2:
                name = '_'.join(name_parts[2:])

        if os.path.isdir(path):
            if len(os.listdir(path)) == 0:
                with open(os.path.join(path, 'leeme.txt'), 'w', encoding='utf-8') as file:
                    file.write("No se añadio información al archivo")
            else:
                for item in [os.path.join(path, c) for c in os.listdir(path)]:
                    if os.path.isdir(item):
                        depth, df_logs = loop_miterd_criteria(main_folder=item, depth=depth + 1, df_logs=df_logs)
        if not os.path.isdir(path):
            file_count = {}
            for file_name in os.listdir(path_):
                name, ext = os.path.splitext(file_name)
                old_path = os.path.join(path_, name + ext)            
            if name in file_count:
                file_count[name] += 1                
                path = os.path.join(path_, name[:max_length - len(ext) - file_count[name] - 1] + "_" + str(file_count[name]) + ext)
            else:
                file_count[name] = 0
                path = os.path.join(path_, name[:max_length - len(ext)] + ext)
            
            if path != old_path:
                os.rename(old_path, path)
            
            
            name, ext = os.path.splitext(path)
            if ext in ['.doc', '.dotx', '.dotm', '.dot', '.docm', '.docb', '.rtf', '.odt', '.wps']:
                print(f'PARSE {ext} - {path}')
                old_path = path
                path = path.replace(ext, '.pdf')
                convertword(filepath = old_path, outputpdf_path = path)
                os.remove(old_path)
                print(f'REMOVING {old_path} PREVIOUSLY {ext}')
            elif ext in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.tif', '.svg', '.webp']:
                pass
            elif ext in ['.lnk']:
                old_path = path
                os.remove(path = old_path)
                print(f'REMOVING {old_path} PREVIOUSLY {ext}')
            elif ext in ['.xls', '.xlsm']:
                # , '.csv', '.ods'
                print(f'PARSE {ext} - {path}')
                old_path = path
                path = path.replace(ext, '.xlsx')
                df = pd.read_excel(old_path, engine='xlrd', header=None)
                df.to_excel(path, index=False, header=None)
                os.remove(path=old_path)
                print(f'REMOVING {old_path} PREVIOUSLY {ext}')
            elif ext in ['.pdf','.txt','.docx','.xlsx']:
                pass
            else:
                print(f'ERROR: {ext}')                 
    
    return z, df_logs

In [ ]:
def fix_miterd_folder_criteria(main_folder):

    # Inicialización de los DataFrames de errores y advertencias
    df_logs = \
        {
            'errors': pd.DataFrame(columns=['description', 'value']),
            'Warning': pd.DataFrame(columns=['description', 'value'])
        }

    # Llamada a la función recursiva comenzando desde el directorio principal con profundidad 1
    df_logs = loop_miterd_criteria(main_folder, df_logs=df_logs, depth=1)
    return df_logs

In [44]:
if __name__ == "__main__":
    folder_path = ".\\"
    logs = fix_miterd_folder_criteria(os.path.join(folder_path, 'Years', '2025-2030'))
    # Guardar los datos en un archivo Excel
    excel_file = 'logs.xlsx'
    with ExcelWriter(excel_file) as writer:
        logs['Warning'].to_excel(writer, sheet_name='Warnings', index=False)
        logs['errors'].to_excel(writer, sheet_name='Errors', index=False)

    print(f"Los datos se han guardado en el archivo '{excel_file}'.")